# Cloud Pak for Data as a Service user management

### Make sure to set the API_key variable in the next cell before executing it

This notebook covers:
- Adding python functions from a zip file
- Getting an access token from an API key
- Listing the members of the account
- Listing available groups
- Listing available roles
- Inviting a user to the account
- Removing a user from the account

References:
- [User management API](https://cloud.ibm.com/apidocs/user-management)
- [Access group API](https://cloud.ibm.com/apidocs/iam-access-groups)
- [Role API](https://cloud.ibm.com/apidocs/iam-policy-management)


In [ ]:
import requests
import json
import warnings
import os, sys
from datetime import datetime
import inspect

import zipfile
from io import BytesIO

API_key = "<Insert API key here>" # from cloud.ibm.com...

# Should not use verify=False but I don't want to deal with SSL
verify=False
warnings.filterwarnings("ignore") # one of "error", "ignore", "always", "default", "module", or "once"

## Support functions
instructions:
- In the next empty cell, use **`Insert to code`**, **`StreamingBody object`** for the file **`cpdalllibs.zip`**
- Make sure to use **`streaming_body_1`** in the line:<br/>
`streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']`

In [ ]:
# Load the python support functions
!rm -rf cpdalllibs
myzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()))

myzip.extractall('.')

sys.path.append(".")
from cpdalllibs.cpdaaslibfns import *
importcpdaas()

# Test if we have access
help(getUsers)
print("\nShow the source of a function:\n")
print(inspect.getsource(getRoles))

## Get an access token

In [ ]:
resp = getToken(API_key)
if resp.status_code > 200 :
    print("getToken status code: {}, reason: {}".format(resp.status_code,resp.reason))
resp_json = resp.json()
access_token = resp_json['access_token']
print("Got a token at {} GMT".format(datetime.now().time().isoformat("seconds")))

# Header to use in subsequent queries
headersAPI = {
        'accept': 'application/json',
        'Content-type': 'application/json',
        'Authorization': 'Bearer ' + access_token,
        'cache-control': 'no-cache'
}
# Get the detail to extract the account_id
resp = apikeyDetails(API_key, access_token)
if resp.status_code > 200 :
    print("apikeyDetails status code: {}, reason: {}".format(resp.status_code,resp.reason))
key_details_json = resp.json()

account_id = key_details_json['account_id']
iam_id = key_details_json['iam_id']

## List the users in the account

In [ ]:
all_users = getUsers(headersAPI, account_id)
print("Total count: {}".format(len(all_users)))
print("\n".join(["{:32} - {}".format(item['user_id'],item['iam_id']) for item in all_users]))

In [ ]:
# See what a member record looks like
print(json.dumps(all_users[0], indent=4))

## List available groups

In [ ]:
all_groups = getAccessGroups(headersAPI, account_id)
print("Total number of access groups: {}".format(len(all_groups)))
print("\n".join(['{:<15} | {}'.format(item['name'],item['id']) for item in all_groups]))

In [ ]:
print(json.dumps(all_groups[0], indent=4))

## List access group members
This is a default group. Everyone is a member of this group so no-one is listed

In [ ]:
members = getGroupMembers(headersAPI, all_groups[0]['id'])
print(json.dumps(members, indent=4))

## Create a group

In [ ]:
group_name = "MyGroup"
group_description = "Temporary group for testing only"
resp = cre8Group(headersAPI, account_id, group_name, group_description)
if resp.status_code > 201 :
    print("cre8Group status code: {}, reason: {}".format(resp.status_code,resp.reason))
new_group = resp.json()
print(json.dumps(new_group, indent=4))

## List available roles
There are three types of roles: custom, service, and system

In [ ]:
all_roles = getRoles(headersAPI)
if len(all_roles['custom_roles']) > 0 :
    print("Total number of custom roles: {}".format(len(all_roles['custom_roles'])))
    print(", ".join([item['display_name'] for item in all_roles['custom_roles']]))
if len(all_roles['service_roles']) > 0 :
    print("Total number of service roles: {}".format(len(all_roles['service_roles'])))
    print(", ".join([item['display_name'] for item in all_roles['service_roles']]))
if len(all_roles['system_roles']) > 0 :
    print("Total number of system roles: {}".format(len(all_roles['system_roles'])))
    print(", ".join([item['display_name'] for item in all_roles['system_roles']]))

In [ ]:
# Show the definition of a system role
print(json.dumps(all_roles['system_roles'][0], indent=4))

## Invite a user to the account
There is no need to have a valid user email. For this exercise, using `user1@company.com` works fine.

Note that we automatically add the user to the new group that was created earlier.

In [ ]:
# https://cloud.ibm.com/apidocs/user-management#invite-users
user_email = "user1@company.com"

data = {
    "users": [
      {
        "email": user_email,
        "account_role": "Member"
      }],
    "access_groups":[
        new_group['id']
    ]
}
resp = inviteUser(headersAPI,account_id, data)
if resp.status_code > 202 :
    print("inviteUser() status code: {}, reason: {}".format(resp.status_code,resp.reason))

user_json = resp.json()['resources']
print("New user definition:")
print(json.dumps(user_json, indent=4))
iam_id = resp.json()['resources'][0]['id']


## List the members of the new group
We should see the new user. The user id is preceded by "BSS-"

In [ ]:
members = getGroupMembers(headersAPI, new_group['id'])
print(json.dumps(members, indent=4))

## Delete a user from the account

In [ ]:
resp = removeUser(headersAPI, iam_id, account_id)
if resp.status_code > 204 : # if error
    print("Status code: {}, reason: {}".format(resp.status_code,resp.reason))
else :
    print("User {}/{} removed".format(user_email,iam_id))

## Delete the new group

In [ ]:
resp = deleteGroup(headersAPI, new_group['id'])
if resp.status_code > 204 : # if error
    print("Status code: {}, reason: {}".format(resp.status_code,resp.reason))
else :
    print("Group {} removed".format(new_group['name']))

### Author
**Jacques Roy** is a member of the IBM Enablement for Data and AI

Copyright © 2023. This notebook and its source code are released under the terms of the MIT License.